CME 538
TIME TO IMPORT ALL THE DATA FROM THE DIFFERENT DATA SOURCES! 

In [1]:
import pandas as pd
import requests
import json
import xml.etree.ElementTree as ET
import API_keys1


In [4]:
# API PULL STATUSES: 
# Vancouver     done! 
# TTC           done! Files downloaded from online database on 2023-11-07
# Go Transit    done!
# STM           in progress - response issues from the STM API 
# OC Transpo    in progress
# 

VANCOUVER

In [34]:
# VANCOUVER - TRANSLINK MANAGED API! 
# https://www.translink.ca/about-us/doing-business-with-translink/app-developer-resources/rtti

def pull_translink_data():
    translink_url = "https://api.translink.ca/rttiapi/v1/buses?apikey="
    
    response = requests.get(translink_url+API_keys1.Translink_API_Key)
    
    translink_Data = response.text
    print(translink_Data)
    
    root = ET.fromstring(translink_Data)

    # Replace 'output.xml' with the desired filename
    output_file = 'translink_data.xml'
    with open(output_file, 'wb') as xml_file:
        xml_file.write(ET.tostring(root, encoding='utf-8'))

    


In [7]:
# pull_translink_data()

<Buses xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><Bus><VehicleNo>12001</VehicleNo><TripId>13539101</TripId><RouteNo>049</RouteNo><Direction>WEST</Direction><Destination>UBC</Destination><Pattern>WB1</Pattern><Latitude>49.222067</Latitude><Longitude>-123.024783</Longitude><RecordedTime>10:38:27 am</RecordedTime><RouteMap><Href>https://nb.translink.ca/geodata/049.kmz</Href></RouteMap></Bus><Bus><VehicleNo>12003</VehicleNo><TripId>13590009</TripId><RouteNo>049</RouteNo><Direction>EAST</Direction><Destination>METROTOWN STN</Destination><Pattern>EB1</Pattern><Latitude>49.247100</Latitude><Longitude>-123.232083</Longitude><RecordedTime>10:38:48 am</RecordedTime><RouteMap><Href>https://nb.translink.ca/geodata/049.kmz</Href></RouteMap></Bus><Bus><VehicleNo>12007</VehicleNo><TripId>13590007</TripId><RouteNo>049</RouteNo><Direction>EAST</Direction><Destination>METROTOWN STN</Destination><Pattern>EB1</Pattern><Latitude>49.226467</Latitude><Longitude>-123.005067</Longitude><RecordedTime>

MONTREAL

In [8]:
# MONTREAL - STM MANAGED API! 

def pull_stm_data():
    stm_url = "https://api.stm.info/pub/od/gtfs-rt/ic/v2"
    
    
    


OTTAWA

In [2]:
# OTTAWA - OC TRANSPO INFO THROUGH TRANSITLAND API! 

def pull_octranspo_data():
    # Documentation: https://www.octranspo.com/en/plan-your-trip/travel-tools/developers/dev-doc
    stops_interrupted = pd.read_excel("OC_Transpo_Stop_List.xlsx", header=0)["Stop_Number"]
    
    start_value = 7649

    # Find the index where the start_value is located
    start_index = stops_interrupted[stops_interrupted == start_value].index[0]
    
    # Filter the Series based on the condition
    stops = stops_interrupted[start_index:].to_list()
    
    for stop in stops:
        data = requests.post(
        'https://api.octranspo1.com/v2.0/GetNextTripsForStopAllRoutes',
        data={
            "appID": API_keys1.ottawa_App_ID,
            "apiKey": API_keys1.ottawa_API_Key,
            "stopNo": stops,
            "format": "json"
        })
        print(stop)
        print(data)
        
        file_path = "octranspo_output.json"
        if "GetRoute" in data.text[2:10]:
            data = data.json()
            with open(file_path, 'r+') as file:
                try:
                    # Load existing data from the file
                    file_data = json.load(file)
                except json.decoder.JSONDecodeError:
                    # If the file is empty, initialize as an empty list
                    file_data = {}
        
                # Append the new data to the existing data
                file_data[stop] = data
        
                # Move the file cursor to the beginning of the file
                file.seek(0)
        
                # Write the updated data back to the file
                json.dump(file_data, file, indent=4)
        
                # Truncate the file at the current position to remove any extra data
                file.truncate()
        else:
            continue
    
    


In [ ]:
pull_octranspo_data()

2757
<Response [200]>
2760
<Response [200]>
1747
<Response [200]>
4042
<Response [200]>
2755
<Response [200]>
2756
<Response [200]>
2761
<Response [200]>
2750
<Response [200]>
2758
<Response [200]>
2752
<Response [200]>
2751
<Response [200]>
2753
<Response [200]>
1746
<Response [200]>
2749
<Response [200]>
5718
<Response [200]>
3656
<Response [200]>
7561
<Response [200]>
7683
<Response [200]>
1368
<Response [200]>
1228
<Response [200]>
1512
<Response [200]>
9884
<Response [200]>
9882
<Response [200]>
9883
<Response [200]>
7091
<Response [200]>
7093
<Response [200]>
7092
<Response [200]>
7328
<Response [200]>
9880
<Response [200]>
9881
<Response [200]>
8580
<Response [200]>
8576
<Response [200]>
7097
<Response [200]>
7094
<Response [200]>
3791
<Response [200]>
4624
<Response [200]>
3790
<Response [200]>
4625
<Response [200]>
3789
<Response [200]>
4287
<Response [200]>
8881
<Response [200]>
8874
<Response [200]>
4286
<Response [200]>
8875
<Response [200]>
8953
<Response [200]>
6598
<Resp

TORONTO

In [24]:
# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

def pull_gotrain_data():
    gotrain_url = "http://api.openmetrolinx.com/OpenDataAPI/api/V1/Gtfs/Feed/TripUpdates?key="
    
    response = requests.get(gotrain_url+API_keys1.Metrolinx_API_Key)
    
    gotrain_Data = response.text
    print(gotrain_Data)
    
    # Specify the file path where you want to save the JSON data
    file_path = "gotrain_output.json"  
    
    # Write the JSON data to the local file
    with open(file_path, "w") as file:
        json.dump(gotrain_Data, file, indent=4)
    
    return

# sample code from TTC, modified to try to pull data 
def test_TTC_Open_Data_Call():
    # To hit our API, you'll be making requests to:
    base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
    
    # Datasets are called "packages". Each package can contain many "resources"
    # To retrieve the metadata for this package and its resources, use the package name in this page's URL:
    url = base_url + "/api/3/action/package_show"
    params = { "id": "ttc-real-time-next-vehicle-arrival-nvas"}
    package = requests.get(url, params = params).json()
    print(package)
    
    # TTC_file_path = "TTC_Subway_output.json"
    
    #with open(TTC_file_path, "w") as file:
        #json.dump(package, file, indent=4)
    
    # To get resource data:
    for idx, resource in enumerate(package["result"]["resources"]):
     
           # To get metadata for non datastore_active resources:
           if not resource["datastore_active"]:
               url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
               resource_metadata = requests.get(url).json()
               print(resource_metadata)
               # From here, you can use the "url" attribute to download this file
            

In [25]:
# call from the APIs
# pull_gotrain_data()



{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=package_show', 'success': True, 'result': {'author': 'opendata@ttc.ca', 'author_email': 'opendata@ttc.ca', 'creator_user_id': '329e1506-b545-4fc7-a4ea-e614f220eea7', 'dataset_category': 'Document', 'date_published': '2019-07-23 18:11:09.685462', 'excerpt': 'The NVAS XML Feeds provide real-time next vehicle arrival information, including: vehicle locations, predicted arrival & departure times, and route definitions.', 'formats': 'PDF', 'id': '8217e43a-bba2-4e6c-82f5-bf6d1a52545d', 'is_retired': 'false', 'isopen': False, 'last_refreshed': '2023-10-03 20:27:28.954583', 'license_id': 'notspecified', 'license_title': 'License not specified', 'maintainer': None, 'maintainer_email': 'opendata@ttc.ca', 'metadata_created': '2022-03-10T22:09:13.468003', 'metadata_modified': '2023-10-03T20:27:29.502028', 'name': 'ttc-real-time-next-vehicle-arrival-nvas', 'notes': '**TTC will be revising their current process and